In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    
    if PARAM['atmpro'] == PARAM['atmpro']:
        atmpro = PARAM['atmpro']
    else:
        atmpro = 'mls'
        
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'h2o': 'atmpro', 'co2': 0.0004}
band = [11]
atmpro = mls
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band9
  atmpro = mls
  band = 9
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [5]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(500, 250)]
  tsfc = 294
  vmax = 3000
  vmin = 1900
  w_diffuse = [(1.66, 1.66, 1.66, 1.66, 1.66)]
  wgt = [(0.4, 0.4, 0.5, 0.55, 0.85)]
o co2 band9
  atmpro = mls
  band = 9
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [3, 3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (50, 250)]
  tsfc = 294
  vmax = 3000
  vmin = 1900
  w_diffuse = [(1.66, 1.66, 1.75), (1.75, 1.6, 1.85)]
  wgt = [(0.7, 0.8, 0.7), (0.8, 0.7, 0.8)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.131813                    1  0.283116
0.000750        2  0.009770                    2  0.097300
0.001052        3  0.006112                    3  0.050897
0.001476        4  0.001071                    4  0.013040
0.002070        5 -0.003875                    5 -0.016500
0.002904        6 -0.008233                    6 -0.030912
0.004074        7 -0.012189                    7 -0.038358
0.005714        8 -0.016012                    8 -0.038887
0.008015        9 -0.019905                    9 -0.040226
0.011243       10 -0.024035                   10 -0.040040
0.015771       11 -0.028514                   11 -0.041137
0.022122       12 -0.033517                   12 -0.043733
0.031031       13 -0.039586                   13 -0.047907
0.043528       14 -0.049764                   14 -0.057964
0.061057       15 -0.061690                   15 -0.068834
0.085645       16 -0.065718                   16 -0.067251
0.120136       17 -0.061404                   17 -0.052327
0.168516       18 -0.056098                   18 -0.034179
0.236378       19 -0.044777                   19 -0.015503
0.331549       20 -0.021935                   20  0.002133
0.465100       21  0.021590                   21  0.025757
0.652400       22  0.119315                   22  0.096955
0.915100       23  0.271593                   23  0.229730
1.283650       24  0.269905                   24  0.239956
1.800600       25  0.151761                   25  0.165169
2.525700       26  0.082714                   26  0.114476
3.542800       27  0.045871                   27  0.070789
4.969550       28  0.024650                   28  0.031673
6.970850       29  0.013194                   29  0.007876
9.778100       30  0.006722                   30  0.003305
13.715850      31  0.002255                   31  0.005145
19.239350      32 -0.000621                   32  0.003276
26.987250      33 -0.001172                   33  0.000614
37.855300      34 -0.000740                   34 -0.000836
53.100050      35 -0.000840                   35 -0.001322
73.887500      36 -0.001263                   36 -0.001621
97.662500      37 -0.001454                   37 -0.001718
121.437500     38 -0.001505                   38 -0.001721
145.212500     39 -0.001962                   39 -0.002234
168.987500     40 -0.002734                   40 -0.003105
192.762500     41 -0.003144                   41 -0.003550
216.537500     42 -0.002945                   42 -0.003420
240.312500     43 -0.002853                   43 -0.003472
264.087500     44 -0.002869                   44 -0.003697
287.862500     45 -0.002938                   45 -0.003801
311.637500     46 -0.002953                   46 -0.003761
335.412500     47 -0.002818                   47 -0.003517
359.187500     48 -0.002524                   48 -0.003361
382.962500     49 -0.002066                   49 -0.003147
406.737500     50 -0.001436                   50 -0.002799
430.512500     51 -0.000659                   51 -0.002190
454.287500     52  0.000214                   52 -0.001390
478.062500     53  0.001190                   53 -0.000060
501.837500     54  0.002291                   54  0.001409
525.612500     55  0.003543                   55  0.003146
549.387500     56  0.004983                   56  0.005076
573.162500     57  0.006667                   57  0.007150
596.937500     58  0.008878                   58  0.009574
620.712500     59  0.011150                   59  0.011768
644.487500     60  0.013407                   60  0.013504
668.262500     61  0.015687                   61  0.014827
692.037500     62  0.018012                   62  0.015481
715.812500     63  0.020436                   63  0.016743
739.587500     64  0.023029                   64  0.018394
763.362500     65  0.025883                   65  0.020541
78

# Fluxes by Layer

CRD                        CLIRAD                \
                       flug      fldg     fnetg      flug          fldg   
pressure    level                                                         
0.000000    1     -4.087705  0.000000 -4.087705 -4.021992  7.693342e-09   
0.000624    2     -4.087709  0.000014 -4.087695 -4.021998  2.764778e-05   
0.000876    3     -4.087711  0.000017 -4.087695 -4.022001  3.353274e-05   
0.001229    4     -4.087714  0.000019 -4.087695 -4.022005  3.962280e-05   
0.001723    5     -4.087718  0.000023 -4.087695 -4.022011  4.557033e-05   
0.002417    6     -4.087722  0.000028 -4.087695 -4.022017  5.113664e-05   
0.003391    7     -4.087729  0.000033 -4.087696 -4.022027  5.674534e-05   
0.004757    8     -4.087738  0.000040 -4.087698 -4.022039  6.301037e-05   
0.006672    9     -4.087750  0.000048 -4.087701 -4.022056  7.099503e-05   
0.009359    10    -4.087766  0.000058 -4.087708 -4.022079  8.141181e-05   
0.013128    11    -4.087789  0.000070 -4.087719 -4.022111  9.568016e-05   
0.018415    12    -4.087821  0.000085 -4.087736 -4.022156  1.151397e-04   
0.025830    13    -4.087868  0.000102 -4.087766 -4.022221  1.416599e-04   
0.036232    14    -4.087937  0.000123 -4.087815 -4.022317  1.777599e-04   
0.050823    15    -4.088048  0.000147 -4.087901 -4.022465  2.260945e-04   
0.071291    16    -4.088236  0.000186 -4.088050 -4.022707  3.012815e-04   
0.100000    17    -4.088552  0.000278 -4.088274 -4.023087  4.525129e-04   
0.140271    18    -4.089052  0.000485 -4.088567 -4.023652  7.679791e-04   
0.196760    19    -4.089828  0.000886 -4.088942 -4.024479  1.366329e-03   
0.275997    20    -4.091009  0.001646 -4.089362 -4.025673  2.414841e-03   
0.387100    21    -4.092733  0.003082 -4.089651 -4.027355  4.124860e-03   
0.543100    22    -4.095036  0.005783 -4.089252 -4.029578  6.823568e-03   
0.761700    23    -4.096979  0.010817 -4.086162 -4.031434  1.119075e-02   
1.068500    24    -4.095148  0.018857 -4.076292 -4.029562  1.766812e-02   
1.498800    25    -4.088866  0.026332 -4.062534 -4.023654  2.399297e-02   
2.102400    26    -4.081636  0.029953 -4.051683 -4.016827  2.897647e-02   
2.949000    27    -4.074688  0.031300 -4.043388 -4.010050  3.368073e-02   
4.136600    28    -4.068312  0.031377 -4.036935 -4.003502  3.709267e-02   
5.802500    29    -4.062654  0.030584 -4.032071 -3.997363  3.720440e-02   
8.139200    30    -4.057630  0.029211 -4.028419 -3.991573  3.359492e-02   
11.417000   31    -4.053099  0.027290 -4.025809 -3.986063  2.936819e-02   
16.014700   32    -4.049231  0.024650 -4.024581 -3.981195  2.730312e-02   
22.464000   33    -4.046531  0.021476 -4.025055 -3.977643  2.625331e-02   
31.510500   34    -4.045026  0.018715 -4.026311 -3.975371  2.463980e-02   
44.200100   35    -4.044336  0.016913 -4.027423 -3.973929  2.194180e-02   
62.000000   36    -4.044823  0.015630 -4.029194 -3.973741  1.896650e-02   
85.775000   37    -4.047482  0.014730 -4.032752 -3.976332  1.699196e-02   
109.550000  38    -4.051487  0.014641 -4.036846 -3.980761  1.658262e-02   
133.325000  39    -4.056301  0.015215 -4.041086 -3.986187  1.716020e-02   
157.100000  40    -4.062470  0.015859 -4.046611 -3.993171  1.785271e-02   
180.875000  41    -4.071322  0.017010 -4.054312 -4.003293  1.922755e-02   
204.650000  42    -4.083426  0.020259 -4.063168 -4.017097  2.303137e-02   
228.425000  43    -4.098349  0.026888 -4.071461 -4.033931  3.023224e-02   
252.200000  44    -4.116266  0.036769 -4.079497 -4.053780  4.030239e-02   
275.975000  45    -4.137862  0.050287 -4.087576 -4.077324  5.343289e-02   
299.750000  46    -4.164083  0.068233 -4.095850 -4.105393  7.079478e-02   
323.525000  47    -4.194618  0.090452 -4.104166 -4.137803  9.260985e-02   
347.300000  48    -4.229378  0.117277 -4.112102 -4.174323  1.192239e-01   
371.075000  49    -4.268599  0.149388 -4.119211 -4.215280  1.507149e-01   
394.850000  50    -4.312458  0.187429 -4.125029 -4.260828  1.873988e-01   
418.625000  51    -4.361049  0.231977 -4.129072 -

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')